# perceptron

In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Load the datasets
df = pd.read_csv('../Datasets/titanic/train.csv')
test_df = pd.read_csv("../Datasets/titanic/test.csv")
y_test_df = pd.read_csv("../Datasets/titanic/gender_submission.csv")

titanic_output_for_perceptron = df['Survived']
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test_df = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
print(df.isna().sum())


titanic_input_for_perceptron = df
titanic_input_for_perceptron["Age"] = titanic_input_for_perceptron["Age"].fillna(titanic_input_for_perceptron["Age"].mean())
test_df["Age"] = test_df["Age"].fillna(test_df["Age"].mean())

titanic_input_for_perceptron["Embarked"] = titanic_input_for_perceptron["Embarked"].fillna(titanic_input_for_perceptron["Embarked"].mode()[0])
test_df["Embarked"] = test_df["Embarked"].fillna(test_df["Embarked"].mode()[0])


test_df["Fare"] = test_df["Fare"].fillna(test_df["Fare"].mean())

# print(titanic_input_for_perceptron.isna().sum())
print(test_df.isna().sum())

titanic_input_for_perceptron["Sex"] = titanic_input_for_perceptron['Sex'].map({"male":0, "female":1})
test_df["Sex"] = test_df['Sex'].map({"male":0, "female":1})

en = OneHotEncoder()

embarked_encoded = en.fit_transform(titanic_input_for_perceptron[['Embarked']])
embarked_encoded_test_df = en.transform(test_df[['Embarked']])

encoded_col_names = en.get_feature_names_out(['Embarked'])

titanic_input_for_perceptron[encoded_col_names] = embarked_encoded.toarray()
test_df[encoded_col_names] = embarked_encoded_test_df.toarray()

titanic_input_for_perceptron.drop('Embarked', axis=1, inplace=True)
test_df.drop('Embarked', axis=1, inplace=True)

scaler = StandardScaler()
columns = titanic_input_for_perceptron.columns  # Save column names before scaling
titanic_input_for_perceptron = pd.DataFrame(scaler.fit_transform(titanic_input_for_perceptron), columns=columns)
test_df = pd.DataFrame(scaler.transform(test_df), columns=columns)




# print(titanic_input_for_perceptron[['Fare']])


Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [10]:
from sklearn.metrics import accuracy_score
import random

learning_rate = 0.01
# w = [] 
epochs = 10
b = 0

# titanic_input_for_perceptron['Bias'] = 1 
# test_df['Bias'] = 1

def calc_perceptron_output(x, w):
    global b
    res = b
    for i, weight in enumerate(w):
        res += weight * x[i]

    return res

def update_weights(y, ye, x, w):
    learning_rate,b
    for i, input_val in enumerate(x):
        w[i] += learning_rate * input_val * (y - ye)
    
    b += learning_rate * (y - ye)
    

def train_perceptron(inputs, outputs, w=None, epochs=100, learning_rate=0.01):
    # Initialize weights if not provided
    if w is None:
        w = [0] * len(inputs.columns)
    
    # Convert outputs to numpy array for clean comparison
    y_true_values = outputs.values.ravel()
    
    for epoch in range(epochs):
        correct = 0
        for idx, row in inputs.iterrows():
            x = row.values
            y_true = y_true_values[idx]  # Now a scalar value
            total = sum(w_i * x_i for w_i, x_i in zip(w, x))
            y_pred = 1 if total > 0 else 0
            
            if y_pred != y_true:
                # Update weights
                for i in range(len(w)):
                    w[i] += learning_rate * x[i] * (y_true - y_pred)
        
        # Optional: Print accuracy per epoch
        if epoch % 10 == 0:
            predictions = [1 if sum(w_i*x_i for w_i, x_i in zip(w, row)) > 0 else 0 
                         for _, row in inputs.iterrows()]
            acc = sum(p == y for p, y in zip(predictions, y_true_values)) / len(y_true_values)
            print(f"Epoch {epoch}: Accuracy = {acc:.2f}")
    
    return w

def predict_y(test_inputs, w):
    pred_y_outputs = []

    for idx, row in test_inputs.iterrows():
        x = row.tolist()  
        total = calc_perceptron_output(x, w)
        y_pred = 1 if total > 0 else 0
        pred_y_outputs.append(y_pred)

    return pred_y_outputs


# train_perceptron(titanic_input_for_perceptron, titanic_output_for_perceptron)


#pred_y = predict_y(test_df)

# print(accuracy_score(y_test_df['Survived'], pred_y))

# print(y_test_df)


# meta-features (combining features practice)

### using titanic dataset from kaggle

In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
# import standardScal

dataset = pd.read_csv('../Datasets/titanic/train.csv')

test_dataset = pd.read_csv('../Datasets/titanic/test.csv')


x_train = dataset.drop('PassengerId', axis=1)

y_train = x_train[['Survived']]

x_train.drop('Name', axis=1, inplace=True)
x_train.drop('Ticket', axis=1, inplace=True)
x_train.drop('Cabin', axis=1, inplace=True)
x_train.drop('Survived', axis=1, inplace=True)
x_train['Sex'] = x_train['Sex'].map({"male": 0, "female": 1})

age_bins = [0, 12, 18, 65, 100]
age_labels = ['child', 'teen', 'adult', 'senior']


x_train['AgeGroup'] = pd.cut(x_train['Age'], bins=age_bins, labels=age_labels)

x_train['AgeGroup'] = x_train['AgeGroup'].cat.codes

fare_bins = [-1, 10, 50, 100, 1000]
fare_labels = ['cheapest', 'economy', 'premium', 'luxury']

x_train['FareGroup'] = pd.cut(x_train['Fare'], bins=fare_bins, labels=fare_labels)

x_train['FareGroup'] = x_train['FareGroup'].cat.codes

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

embarked_encoded = encoder.fit_transform(x_train[['Embarked']])

encoded_cols = encoder.get_feature_names_out(['Embarked'])
x_train[encoded_cols] = embarked_encoded

x_train.drop(['Embarked', 'Embarked_nan', 'Fare'], axis=1, inplace=True)


x_train['Age'].fillna(x_train['Age'].mean(), inplace=True)
print(x_train.isnull().sum())

x_train['FamilySize'] = x_train['SibSp'] + x_train['Parch'] + 1  # +1 for self

# Bin into categories
x_train['FamilyType'] = pd.cut(x_train['FamilySize'], bins=[0, 1, 4, 11], labels=['Solo', 'Small', 'Large'])

family_dummies = pd.get_dummies(x_train['FamilyType'], prefix='Family')
x_train = pd.concat([x_train, family_dummies], axis=1)
x_train.drop(['Parch', 'SibSp', 'FamilySize', 'FamilyType'], axis=1, inplace=True)

print(x_train.columns)


Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
AgeGroup      0
FareGroup     0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64
Index(['Pclass', 'Sex', 'Age', 'AgeGroup', 'FareGroup', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Family_Solo', 'Family_Small',
       'Family_Large'],
      dtype='object')


/tmp/ipykernel_18011/1614327377.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_train['Age'].fillna(x_train['Age'].mean(), inplace=True)


In [12]:
x_test = test_dataset.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

x_test['Sex'] = x_test['Sex'].map({"male": 0, "female": 1})

x_test['AgeGroup'] = pd.cut(x_test['Age'], bins=age_bins, labels=age_labels)
x_test['AgeGroup'] = x_test['AgeGroup'].cat.codes

x_test['Fare'] = x_test['Fare'].fillna(x_test['Fare'].median())  # Fill missing with median
x_test['FareGroup'] = pd.cut(x_test['Fare'], bins=fare_bins, labels=fare_labels)

x_test['Embarked'] = x_test['Embarked'].fillna(x_test['Embarked'].mode()[0])
embarked_encoded_test = encoder.transform(x_test[['Embarked']])  # Use transform(), not fit_transform()
x_test[encoded_cols] = embarked_encoded_test
x_test.drop('Embarked', axis=1, inplace=True)

x_test['FamilySize'] = x_test['SibSp'] + x_test['Parch'] + 1  # +1 for self
x_test['FareGroup'] = x_test['FareGroup'].cat.codes

# Bin into categories
x_test['FamilyType'] = pd.cut(x_test['FamilySize'], bins=[0, 1, 4, 11], labels=['Solo', 'Small', 'Large'])
family_dummies = pd.get_dummies(x_test['FamilyType'], prefix='Family')
x_test = pd.concat([x_test, family_dummies], axis=1)

x_test.drop(['Parch', 'SibSp', 'FamilySize', 'FamilyType'], axis=1, inplace=True)

missing_cols = set(x_train.columns) - set(x_test.columns)
for col in missing_cols:
    x_test[col] = 0 

x_test = x_test[x_train.columns] 

x_test['Age'].fillna(x_train['Age'].mean(), inplace=True)




# print("\nTest Data Columns:")
print(x_test.columns)
# print("\nMissing Values in Test Data:")
# print(x_test.isnull().sum())


Index(['Pclass', 'Sex', 'Age', 'AgeGroup', 'FareGroup', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Family_Solo', 'Family_Small',
       'Family_Large'],
      dtype='object')


/tmp/ipykernel_18011/3085704054.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_test['Age'].fillna(x_train['Age'].mean(), inplace=True)


In [14]:
w = train_perceptron(x_train, y_train, epochs=200)

Epoch 0: Accuracy = 0.65
Epoch 10: Accuracy = 0.66
Epoch 20: Accuracy = 0.70
Epoch 30: Accuracy = 0.75
Epoch 40: Accuracy = 0.74
Epoch 50: Accuracy = 0.75
Epoch 60: Accuracy = 0.81
Epoch 70: Accuracy = 0.81
Epoch 80: Accuracy = 0.81
Epoch 90: Accuracy = 0.78
Epoch 100: Accuracy = 0.81
Epoch 110: Accuracy = 0.82
Epoch 120: Accuracy = 0.81
Epoch 130: Accuracy = 0.82
Epoch 140: Accuracy = 0.81
Epoch 150: Accuracy = 0.81
Epoch 160: Accuracy = 0.79
Epoch 170: Accuracy = 0.81
Epoch 180: Accuracy = 0.81
Epoch 190: Accuracy = 0.82


In [ ]:
predictions = predict_y(x_test, w)
predictions_train = predict_y(x_train, w)
print(predictions)



y_test = pd.read_csv('../Datasets/titanic/gender_submission.csv')


# print(y_train)
accuracy_score(y_test['Survived'],predictions)
accuracy_score(y_train['Survived'],predictions_train)



[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

0.9330143540669856

In [22]:
import pandas as pd

predictions = predict_y(x_test, w) 



# 2. Create a DataFrame with IDs and predictions
submission = pd.DataFrame({
    'PassengerId': test_dataset['PassengerId'],
    'Survived': predictions
})

submission.to_csv('../submissions/titanic_submission_v2_perceptron.csv', index=False)